In [1]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit_optimization.algorithms.admm_optimizer import ADMMParameters, ADMMOptimizer

import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

from src.ADMM_MIP import *

import warnings
warnings.simplefilter('ignore')

In [2]:
pb_num = 0

df_kp = pd.read_csv(f'instances/kp/P0{pb_num}.csv', header=0)
print(df_kp)

   size  capacity  weights  profits  solution
0   5.0      80.0       15       33         1
1   NaN       NaN       20       24         1
2   NaN       NaN       17       36         1
3   NaN       NaN        8       37         1
4   NaN       NaN       31       12         0


In [3]:
n_kp = int(df_kp.at[0, 'size'])

mdl = QuadraticProgram('Knapsack Problem')

# binary variables
for i in range(n_kp):
    mdl.binary_var(name = f'x_{i}')
    #mdl.continuous_var(lowerbound=0, upperbound=1, name = f'x_{i}')

# objective function
mdl.maximize(linear={(f'x_{i}'):df_kp.loc[i,'profits'] for i in range(n_kp)})

# le constraints
mdl.linear_constraint(linear={f'x_{i}':df_kp.loc[i,'weights'] for i in range(n_kp)}, sense='<=', rhs=df_kp.loc[0,'capacity'])

print(mdl.prettyprint())

Problem name: Knapsack Problem

Maximize
  33*x_0 + 24*x_1 + 36*x_2 + 37*x_3 + 12*x_4

Subject to
  Linear constraints (1)
    15*x_0 + 20*x_1 + 17*x_2 + 8*x_3 + 31*x_4 <= 80  'c0'

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4



In [4]:
ADMMxClassical = ADMMxClassical_Optimizer(mdl)

n_bin_vars = ADMMxClassical.n_bin_vars
n_cont_vars = ADMMxClassical.n_cont_vars
n_eq = ADMMxClassical.n_eq
n_le = ADMMxClassical.n_le
n_ge = ADMMxClassical.n_ge

print(
        '-'*50,
        '[Problem Info]',
        '-Decision Variables',
        f'   number of binary vars: {n_bin_vars}',
        f'   number of continuous vars: {n_cont_vars}',
        '-Linear Constraints',
        f'   number of eq constraints: {n_eq}',
        f'   number of le constraints: {n_le}',
        f'   number of ge constraints: {n_ge}',
        '-'*50,
        sep='\n'
    )

--------------------------------------------------
[Problem Info]
-Decision Variables
   number of binary vars: 5
   number of continuous vars: 0
-Linear Constraints
   number of eq constraints: 0
   number of le constraints: 1
   number of ge constraints: 0
--------------------------------------------------


In [5]:
cplex_result = CplexOptimizer().solve(mdl)
opt_val = cplex_result.fval
x_sol = cplex_result.x

n_bin = mdl.get_num_binary_vars()

print(
        '-'*80,
        '[CPLEX Result]',
        f'   optimal value: {cplex_result.fval:.3f}',
        f'   optimal solution (binary): {x_sol[:n_bin].astype(int)}',
        f'   optimal solution (continuous): {x_sol[n_bin:]}',
        f'   status: {cplex_result.status.name}',
        '-'*80,
        sep='\n'
    )

--------------------------------------------------------------------------------
[CPLEX Result]
   optimal value: 130.000
   optimal solution (binary): [1 1 1 1 0]
   optimal solution (continuous): []
   status: SUCCESS
--------------------------------------------------------------------------------


In [6]:
ADMMxClassical = ADMMxClassical_Optimizer(mdl)

step_max = 5
n_lin_const = ADMMxClassical.n_lin_const
x_ini = np.array([0.5]*(n_bin_vars+n_cont_vars))
lam_ini = np.full(n_lin_const, 30.)
rho = 1.

admm_result = ADMMxClassical.solve(step_max, x_ini, lam_ini, rho)
x_sol = admm_result.x
n_bin = mdl.get_num_binary_vars()

pd.options.display.float_format = '{:.3f}'.format
print(
        '-'*100,
        '[ADMMxClassic Result]',
        f'   optimal value: {admm_result.fval:.3f}',
        f'   optimal solution (binary): {x_sol[:n_bin]}',
        f'   optimal solution (continuous): {x_sol[n_bin:]}',
        f'   status: {admm_result.status.name}',
        '-'*100,
        ADMMxClassical.opt_record,
        '-'*100,
        sep='\n'
    )

----------------------------------------------------------------------------------------------------
[ADMMxClassic Result]
   optimal value: -130.000
   optimal solution (binary): [1. 1. 1. 1. 0.]
   optimal solution (continuous): []
   status: SUCCESS
----------------------------------------------------------------------------------------------------
        objective function value      status
step 0                   -71.000    feasible
step 1                  -142.000  infeasible
step 2                  -130.000    feasible
step 3                  -130.000    feasible
step 4                  -130.000    feasible
step 5                  -130.000    feasible
----------------------------------------------------------------------------------------------------


In [7]:
admm_params = ADMMParameters(
    rho_initial=10, beta=10, factor_c=10, maxiter=100, three_block=True, tol=1.0e-6
)

# initialize ADMM with classical QUBO and convex optimizer
admm = ADMMOptimizer(
    params=admm_params, qubo_optimizer=CplexOptimizer(), continuous_optimizer=CplexOptimizer()
)

# run ADMM to solve problem
admm3b_result = admm.solve(mdl)
print(admm3b_result.prettyprint())

data = admm3b_result.state.residuals

print('-'*80)
for step in range(len(data)):
    print(f'step = {step} \t err = {data[step]}')

objective function value: 142.0
variable values: x_0=1.0, x_1=1.0, x_2=1.0, x_3=1.0, x_4=1.0
status: INFEASIBLE
--------------------------------------------------------------------------------
step = 0 	 err = 0.1249032648077739
step = 1 	 err = 0.05947774492085275
step = 2 	 err = 0.026913006553651878
step = 3 	 err = 0.011545691191349744
step = 4 	 err = 0.0046855610185523875
step = 5 	 err = 0.0017948835405866943
step = 6 	 err = 0.0006476074558519577
step = 7 	 err = 0.00021962339709344947
step = 8 	 err = 6.986390766794455e-05
step = 9 	 err = 2.080571994364556e-05
step = 10 	 err = 5.789487040681808e-06
step = 11 	 err = 1.5028355220712102e-06
step = 12 	 err = 3.629247730908049e-07
